<a href="https://colab.research.google.com/github/Movya777/EDA_and_Feature_Engineering/blob/main/Flight_price_prediction_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# FLight Price Prediction ✈

Aim: Get the data ready for modelling using Exploratory Data Analysis (EDA) and Feature Engineering

In [1]:
from io import IncrementalNewlineDecoder
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

#Read the files - Training

In [2]:
df1=pd.read_excel('Data_Train.xlsx')
df2=pd.read_excel('Test_set.xlsx')

In [3]:
df1.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info,Price
0,IndiGo,24/03/2019,Banglore,New Delhi,BLR → DEL,22:20,01:10 22 Mar,2h 50m,non-stop,No info,3897
1,Air India,1/05/2019,Kolkata,Banglore,CCU → IXR → BBI → BLR,05:50,13:15,7h 25m,2 stops,No info,7662
2,Jet Airways,9/06/2019,Delhi,Cochin,DEL → LKO → BOM → COK,09:25,04:25 10 Jun,19h,2 stops,No info,13882
3,IndiGo,12/05/2019,Kolkata,Banglore,CCU → NAG → BLR,18:05,23:30,5h 25m,1 stop,No info,6218
4,IndiGo,01/03/2019,Banglore,New Delhi,BLR → NAG → DEL,16:50,21:35,4h 45m,1 stop,No info,13302


"Price" is the target variable

In [7]:
df2.head()

,Airline,Date_of_Journey,Source,Destination,Route,Dep_Time,Arrival_Time,Duration,Total_Stops,Additional_Info
0,Jet Airways,6/06/2019,Delhi,Cochin,DEL → BOM → COK,17:30,04:25 07 Jun,10h 55m,1 stop,No info
1,IndiGo,12/05/2019,Kolkata,Banglore,CCU → MAA → BLR,06:20,10:20,4h,1 stop,No info
2,Jet Airways,21/05/2019,Delhi,Cochin,DEL → BOM → COK,19:15,19:00 22 May,23h 45m,1 stop,In-flight meal not included
3,Multiple carriers,21/05/2019,Delhi,Cochin,DEL → BOM → COK,08:00,21:00,13h,1 stop,No info
4,Air Asia,24/06/2019,Banglore,Delhi,BLR → DEL,23:55,02:45 25 Jun,2h 50m,non-stop,No info


WOAHHH!!!!

The dataset looks interesting

Right now, the "Route","Duration" columns looks challenging to me and excites me to dive into analysising it.

Lets dig in... 🔍🔍🔍

In [4]:
print("The length of training dataset is:", len(df1))
print("The length of testing dataset is:", len(df2))

The length of training dataset is: 10683
The length of testing dataset is: 2671


# Merge train and test datasets for pre-processing

In [5]:
df=pd.concat([df1,df2])
print("The length of combined dataset is:", len(df))

The length of combined dataset is: 13354


# Feature Engineering



*   Date_of_Journey



In [6]:
# Date_Of_Journey
df['Date']=df['Date_of_Journey'].str.split('/').str[0].astype(int)
df['Month']=df['Date_of_Journey'].str.split('/').str[1].astype(int)
df['Year']=df['Date_of_Journey'].str.split('/').str[2].astype(int)

In [7]:
df.drop('Date_of_Journey',axis=1,inplace=True)



*   Arrival Time



I just need time and not the information about date (since we have seperate columns for that)

In [8]:
df['Arrival_Time']=df['Arrival_Time'].str.split(' ').str[0]

In [9]:
# Take out the hour and minute from arival time
df['Arival_Hour']=df['Arrival_Time'].str.split(':').str[0].astype(int)
df['Arival_Min']=df['Arrival_Time'].str.split(':').str[1].astype(int)

In [10]:
df.drop('Arrival_Time',axis=1,inplace=True)



*   Dep_Time



In [11]:
df['Dep_Hour']=df['Dep_Time'].str.split(':').str[0].astype(int)
df['Dep_Min']=df['Dep_Time'].str.split(':').str[1].astype(int)

In [12]:
df.drop('Dep_Time',axis=1,inplace=True)



*   Total_Stops



In [13]:
df[df['Total_Stops'].isnull()]

,Airline,Source,Destination,Route,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arival_Hour,Arival_Min,Dep_Hour,Dep_Min
9039,Air India,Delhi,Cochin,NaN,23h 40m,NaN,No info,7480.0,6,5,2019,9,25,9,45


Since we donot have information on the route, lets go with the mode method

In [14]:
dfx=df[(df['Source']=='Delhi') & (df['Destination']=='Cochin')]
dfx['Total_Stops'].value_counts()

,count
Total_Stops,
1 stop,4015
2 stops,1373
non-stop,262
3 stops,31


1 stop is more frequent between Delhi and Cochin. So lets fill the null value with 1

In [15]:
df['Total_Stops'].unique()

array(['non-stop', '2 stops', '1 stop', '3 stops', nan, '4 stops'],
      dtype=object)

In [16]:
df['Total_Stops']=df['Total_Stops'].map({"non-stop":0,"1 stop":1,"2 stops":2,"3 stops":3,"4 stops":4,"nan":1})



*  Route

We can now remove Route column, since most of its information is present in other columns



In [17]:
df.drop('Route',axis=1,inplace=True)

In [18]:
df.head()

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arival_Hour,Arival_Min,Dep_Hour,Dep_Min
0,IndiGo,Banglore,New Delhi,2h 50m,0.0,No info,3897.0,24,3,2019,1,10,22,20
1,Air India,Kolkata,Banglore,7h 25m,2.0,No info,7662.0,1,5,2019,13,15,5,50
2,Jet Airways,Delhi,Cochin,19h,2.0,No info,13882.0,9,6,2019,4,25,9,25
3,IndiGo,Kolkata,Banglore,5h 25m,1.0,No info,6218.0,12,5,2019,23,30,18,5
4,IndiGo,Banglore,New Delhi,4h 45m,1.0,No info,13302.0,1,3,2019,21,35,16,50




*   Duration



My take is to convert into minutes

In [19]:
df['Hour']=df['Duration'].str.split(' ').str[0].str.replace("h","")
df['Min']=df['Duration'].str.split(' ').str[1].str.replace("m","")

In [20]:
df[df['Hour']=="5m"]

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arival_Hour,Arival_Min,Dep_Hour,Dep_Min,Hour,Min
6474,Air India,Mumbai,Hyderabad,5m,2.0,No info,17327.0,6,3,2019,16,55,16,50,5m,NaN
2660,Air India,Mumbai,Hyderabad,5m,2.0,No info,NaN,12,3,2019,16,55,16,50,5m,NaN


😶😶😶 5min?? from Mumbai to Hyderbad.

The above 2 records looks misleading. Lets drop them

In [21]:
df.drop(6474,axis=0,inplace=True)
df.drop(2660,axis=0,inplace=True)

In [22]:
df[df['Hour']=="5m"] # successfully dropped the respective rows

,Airline,Source,Destination,Duration,Total_Stops,Additional_Info,Price,Date,Month,Year,Arival_Hour,Arival_Min,Dep_Hour,Dep_Min,Hour,Min


In [23]:
df['Hour']=df['Hour'].astype(int)

In [26]:
#fill nan in Min column with 0
df['Min']=df['Min'].fillna(0)

In [28]:
df['Min']=df['Min'].astype(int)

In [33]:
# Convert hours into minutes - Multiply Hour column with 60
df['Total_Duration']=df['Hour']*60+df['Min']

In [35]:
#drop the columns
df.drop(['Hour','Min','Duration'],axis=1,inplace=True)



*   Airline, Source, Destination, Additional_Info



In [38]:
df['Airline'].unique()

array(['IndiGo', 'Air India', 'Jet Airways', 'SpiceJet',
       'Multiple carriers', 'GoAir', 'Vistara', 'Air Asia',
       'Vistara Premium economy', 'Jet Airways Business',
       'Multiple carriers Premium economy', 'Trujet'], dtype=object)

Lets do label encoding for the Airline column

We can also do one-hot enconding

In [40]:
from sklearn.preprocessing import LabelEncoder
LabelEncoder=LabelEncoder()

In [41]:
df['Airline']=LabelEncoder.fit_transform(df['Airline'])
df['Source']=LabelEncoder.fit_transform(df['Source'])
df['Destination']=LabelEncoder.fit_transform(df['Destination'])
df['Additional_Info']=LabelEncoder.fit_transform(df['Additional_Info'])

In [43]:
df.shape

(13351, 14)

14 columns...ok, thats good to go

In [44]:
df.head()

,Airline,Source,Destination,Total_Stops,Additional_Info,Price,Date,Month,Year,Arival_Hour,Arival_Min,Dep_Hour,Dep_Min,Total_Duration
0,3,0,5,0.0,8,3897.0,24,3,2019,1,10,22,20,170
1,1,3,0,2.0,8,7662.0,1,5,2019,13,15,5,50,445
2,4,2,1,2.0,8,13882.0,9,6,2019,4,25,9,25,1140
3,3,3,0,1.0,8,6218.0,12,5,2019,23,30,18,5,325
4,3,0,5,1.0,8,13302.0,1,3,2019,21,35,16,50,285


HORRAY!!! 🙂🙂🙂

The data is ready for modelling